In [1]:
from src.genre_classification.F_Basic_Models import Basic_Models
import pandas as pd

In [2]:
train_data = pd.read_csv(r'.\datasets\dataset_train.csv')
test_data = pd.read_csv(r'.\datasets\dataset_test.csv')


In [ ]:
rn = Basic_Models(model_type= "Random_Forest")

rn.fit(train_data)

print("dsjfds")

y_hat, y_test = rn.transform(test_data)

print(y_hat, y_test)

Modelo de spaCy 'en_core_web_sm' cargado.
